# Classification with FACET

***

FACET is composed of the following key components:

- **Model Inspection**

    FACET introduces a new algorithm to quantify dependencies and interactions between features in ML models. This new tool for human-explainable AI adds a new, global perspective to the observation-level explanations provided by the popular [SHAP](https://shap.readthedocs.io/en/latest/) approach. To learn more about FACET's model inspection capabilities, see the getting started example below.


- **Model Simulation**

    FACET's model simulation algorithms use ML models for *virtual experiments* to help identify scenarios that optimise predicted  outcomes. To quantify the uncertainty in simulations, FACET utilises a range of bootstrapping algorithms including stationary and stratified bootstraps. For an example of FACET’s bootstrap simulations, see the getting started example below.    
    
    
- **Enhanced Machine Learning Workflow**  

    FACET offers an efficient and transparent machine learning workflow, enhancing [scikit-learn]( https://scikit-learn.org/stable/index.html)'s tried and tested pipelining paradigm with new capabilities for model selection, inspection, and simulation. FACET also introduces [sklearndf](https://github.com/BCG-Gamma/sklearndf), an augmented version of *scikit-learn* with enhanced support for *pandas* dataframes that ensures end-to-end traceability of features.    

***

**Context**

Utilizing FACET, we will do the following:

1. create a pipeline to find identify a well-performing classifier.
2. perform model inspection and simulation to gain understanding and insight into key factors

***

**Tutorial outline**

1. [Required imports](#Required-imports)
2. [Preprocessing and initial feature selection](#Preprocessing-and-initial-feature-selection)
3. [Selecting a learner using FACET ranker](#Selecting-a-learner-using-FACET-ranker)
4. [Using FACET for advanced model inspection](#Using-FACET-for-advanced-model-inspection)
5. [Summary](#Summary)

In [ ]:
# this cell's metadata contains
# "nbsphinx": "hidden" so it is hidden by nbsphinx


# ignore irrelevant warnings that would affect the output of this tutorial notebook

import warnings
import tableone # need to import this to suppress an IPython warning triggered by tableone

warnings.filterwarnings("ignore", category=UserWarning, message=r".*Xcode_8\.3\.3")
warnings.filterwarnings("ignore", message=r".*`should_run_async` will not call `transform_cell`")
warnings.filterwarnings("ignore", message=r".*`np\..*` is a deprecated alias")


# set global options for matplotlib

import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams["figure.figsize"] = (16.0, 8.0)
matplotlib.rcParams["figure.dpi"] = 72

# Required imports

In order to run this notebook, we will import not only the FACET package, but also other packages useful to solve this task. Overall, we can break down the imports into three categories: 

1. Common packages (pandas, matplotlib, etc.)
2. Required FACET classes (inspection, selection, validation, simulation, etc.)

**Common package imports**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap
import seaborn as sns
import tableone
from sklearn.compose import make_column_selector
from sklearn.model_selection import RepeatedKFold

**FACET imports**

In [ ]:
from facet.data import Sample
from facet.inspection import LearnerInspector
from facet.selection import LearnerRanker, LearnerGrid
from facet.validation import BootstrapCV
from facet.data.partition import ContinuousRangePartitioner
from facet.simulation import UnivariateProbabilitySimulator
from facet.simulation.viz import SimulationDrawer
from facet.crossfit import LearnerCrossfit

**sklearndf imports**

Instead of using the "regular" scikit-learn package, we are going to use sklearndf (see on [GitHub](https://github.com/orgs/BCG-Gamma/sklearndf/)). sklearndf is an open source library designed to address a common issue with scikit-learn: the outputs of transformers are numpy arrays, even when the input is a data frame. However, to inspect a model it is essential to keep track of the feature names. sklearndf retains all the functionality available through scikit-learn plus the feature traceability and usability associated with Pandas data frames. Additionally, the names of all your favourite scikit-learn functions are the same except for `DF` on the end. For example, the standard scikit-learn import:

`from sklearn.pipeline import Pipeline`

becomes:

`from sklearndf.pipeline import PipelineDF`

In [ ]:
from sklearndf.pipeline import PipelineDF, ClassifierPipelineDF
from sklearndf.classification import RandomForestClassifierDF
from sklearndf.classification.extra import LGBMClassifierDF
from sklearndf.transformation import (
    ColumnTransformerDF,
    OneHotEncoderDF,
    SimpleImputerDF,
)
from sklearndf.transformation.extra import BorutaDF

**pytools imports**

pytools (see on [GitHub](https://github.com/BCG-Gamma/pytools)) is an open source library containing general machine learning and visualization utilities, some of which are useful for visualising the advanced model inspection capabilities of FACET.

In [ ]:
from pytools.viz.dendrogram import DendrogramDrawer, LinkageTree
from pytools.viz.matrix import MatrixDrawer

# Preprocessing and initial feature selection

First we need to load our data and create a simple preprocessing pipeline.

In [ ]:
# load the prepared data frame
shop_df = pd.read_csv("online_shopers_intention.csv")
# have a look
shop_df.head()

In [ ]:
# to ensure a quick run we will use a random sample of 1000 observations
shop_df = shop_df.sample(n=1000, random_state=42)

For easier data management we will create a sample object using FACET's `Sample` class, which allows us to: 

- Quickly access the target vs. features
- Pass our data into sklearndf pipelines
- Pass information to other FACET functions

In [ ]:
# create a FACET sample object
shop = Sample(
    observations=shop_df,
    feature_names=shop_df.drop(columns=["Revenue"]).columns,
    target_name="Revenue",
)

Next we create a minimum preprocessing pipeline which based on our initial EDA

1. Simple imputation for missing values in both continuous and categorical features
2. One-hot encoding for categorical features

We will use the sklearndf wrappers for scikit-learn functions such as `SimpleImputerDF` in place of `SimpleImputer`, `OneHotEncoderDF` in place of `OneHotEncoder`, and so on.

In [ ]:
# for categorical features we will use the mode as the imputation value and also one-hot encode
preprocessing_categorical = PipelineDF(
    steps=[
        ("imputer", SimpleImputerDF(strategy="most_frequent", fill_value="<na>")),
        ("one-hot", OneHotEncoderDF(sparse=False, handle_unknown="ignore")),
    ]
)

# for numeric features we will impute using the median
preprocessing_numerical = SimpleImputerDF(strategy="median")

# put the pipeline together
preprocessing_features = ColumnTransformerDF(
    transformers=[
        (
            "categorical",
            preprocessing_categorical,
            make_column_selector(dtype_include=object),
        ),
        (
            "numerical",
            preprocessing_numerical,
            make_column_selector(dtype_include=np.number),
        ),
    ]
)

Next, we perform some initial feature selection using Boruta, a recent approach shown to have quite good performance. The Boruta algorithm removes features that are no more predictive than random noise. If you are interested further, please see this  [article](https://www.jstatsoft.org/article/view/v036i11).

The `BorutaDF` transformer in our sklearndf package provides easy access to this powerful method. The approach relies on a tree-based learner, usually a random forest. For settings, a `max_depth` of between 3 and 7 is typically recommended, and here we rely on the default setting of 5. However, as this depends on the number of features and the complexity of interactions one could also explore the sensitivity of feature selection to this parameter. The number of trees is automatically managed by the Boruta feature selector argument `n_estimators="auto"`.

We also use parallelization for the random forest using `n_jobs` to accelerate the Boruta iterations.

In [ ]:
# create the pipeline for Boruta
boruta_feature_selection = PipelineDF(
    steps=[
        ("preprocessing", preprocessing_features),
        (
            "boruta",
            BorutaDF(
                estimator=RandomForestClassifierDF(
                    max_depth=5, n_jobs=-3, random_state=42
                ),
                n_estimators="auto",
                random_state=42,
                verbose=False,
            ),
        ),
    ]
)

# run feature selection using Boruta and report those selected
boruta_feature_selection.fit(X=ahop.features, y=shop.target)
selected = boruta_feature_selection.feature_names_original_.unique()
selected

Boruta identified 10 features (out of a potential 47) that we will retain in our FACET sample object for classification. Note that this feature selection process could be included in a general preprocessing pipeline, however due to the computation involved, we have utilized Boruta here as an initial one-off processing step to narrow down the features for our classifier development.

In [ ]:
# update FACET sample object to only those features Boruta identified as useful
shop_initial_features = shop.keep(feature_names=selected)

# Selecting a learner using FACET ranker

FACET implements several additional useful wrappers which further simplify comparing and tuning a larger number of models and configurations: 

- `LearnerGrid`: allows you to pass a learner pipeline (i.e., classifier + any preprocessing) and a set of hyperparameters
- `LearnerRanker`: multiple LearnerGrids can be passed into this class as a list - this allows tuning hyperparameters both across different types of learners in a single step and ranks the resulting models accordingly

The following learners and hyperparameter ranges will be assessed using 10 repeated 5-fold cross-validation:


1. **Random forest**: with hyperparameters
    - max_depth: [4, 5, 6]
    - min_samples_leaf: [8, 11, 15]  
  
  
2. **Light gradient boosting**: with hyperparameters
    - max_depth: [4, 5, 6]
    - min_samples_leaf: [8, 11, 15]  

Note if you want to see a list of hyperparameters you can use `classifier_name().get_params().keys()` where `classifier_name` could be for example `RandomForestClassifierDF` and if you want to see the default values, just use `classifier_name().get_params()`.

Finally, for this exercise we will use AUC as the performance metric for scoring and ranking our classifiers (the default is accuracy). Note that ranking uses the average performance minus two times the standard deviation, so that we consider both the average performance and variability when selecting a classifier.

First, we specify the classifiers we want to train using `ClassifierPipelineDF` from sklearndf. Note here we also include the feature preprocessing steps we created earlier.

In [ ]:
# random forest learner
rforest_clf = ClassifierPipelineDF(
    preprocessing=preprocessing_features,
    classifier=RandomForestClassifierDF(random_state=42),
)

# light gradient boosting learner
lgbm_clf = ClassifierPipelineDF(
    preprocessing=preprocessing_features,
    classifier=LGBMClassifierDF(random_state=42)
)

Then we create a list of learner grids where each learner grid is created using `LearnerGrid` and allows us to associate a `ClassifierPipelineDF` with a specified set of hyperparameter via the `learner_parameters` argument. Note this structure allows us to easily include additional classifiers and hyperparameters.

In [ ]:
classifier_grid = [
    LearnerGrid(
        pipeline=rforest_clf,
        learner_parameters={
            "max_depth": [4, 5, 6], 
            "min_samples_leaf": [8, 11, 15],
        },
    ),
    LearnerGrid(
        pipeline=lgbm_clf,
        learner_parameters={
            "max_depth": [4, 5, 6],
            "min_samples_leaf": [8, 11, 15],
        },
    ),
]

We now fit the grid defined above using the `LeanerRanker`, which will run a gridsearch (or random search if defined) using 10 repeated 5-fold cross-validation on our selected set of features from Boruta.

In [ ]:
clf_ranker = LearnerRanker(
    grids=classifier_grid,
    cv=RepeatedKFold(n_splits=5, n_repeats=10, random_state=42),
    n_jobs=-3,
    scoring="roc_auc",
).fit(shop_initial_features)

We can see how each model scored using the `summary_report()` method of the `LearnerRanker`.

In [ ]:
# let's look at performance for the top ranked classifiers
clf_ranker.summary_report()

We can see based on our learner ranker, we have selected a Random Forest algorithm that achieved a mean ROC AUC of 0.72 with a SD of 0.03.

# Using FACET for advanced model inspection

The [SHAP approach](http://papers.nips.cc/paper/7062-a-unified-approach-to-interpreting-model-predictions) has become the standard method for model inspection. SHAP values are used to explain the additive contribution of each feature to the prediction for each observation (i.e., explain **individual** predictions).

The FACET `LearnerInspector` computes SHAP values for each crossfit (i.e., a CV fold or bootstrap resample) using the best model identified by the `LearnerRanker`. The FACET `LearnerInspector` then provides advanced model inspection through new SHAP-based summary metrics for understanding pairwise feature redundancy and synergy. Redundancy and synergy are calculated using a new algorithm to understand model predictions from a **global perspective** to complement local SHAP.

The definitions of synergy and redundancy are as follows:

- **Synergy**

  The degree to which the model combines information from one feature with 
  another to predict the target. For example, let's assume we are predicting 
  cardiovascular health using age and gender and the fitted model includes 
  a complex interaction between them. This means these two features are 
  synergistic for predicting cardiovascular health. Further, both features 
  are important to the model and removing either one would significantly 
  impact performance. Let's assume age brings more information to the joint
  contribution than gender. This asymmetric contribution means the synergy for
  (age, gender) is less than the synergy for (gender, age). To think about it
  another way, imagine the prediction is a coordinate you are trying to reach.
  From your starting point, age gets you much closer to this point than 
  gender, however, you need both to get there. Synergy reflects the fact 
  that gender gets more help from age (higher synergy from the perspective 
  of gender) than age does from gender (lower synergy from the perspective of
  age) to reach the prediction. *This leads to an important point: synergy 
  is a naturally asymmetric property of the global information two interacting 
  features contribute to the model predictions.* Synergy is expressed as a 
  percentage ranging from 0% (full autonomy) to 100% (full synergy).


- **Redundancy**

  The degree to which a feature in a model duplicates the information of a 
  second feature to predict the target. For example, let's assume we had 
  house size and number of bedrooms for predicting house price. These 
  features capture similar information as the more bedrooms the larger 
  the house and likely a higher price on average. The redundancy for 
  (number of bedrooms, house size) will be greater than the redundancy 
  for (house size, number of bedrooms). This is because house size 
  "knows" more of what number of bedrooms does for predicting house price 
  than vice-versa. Hence, there is greater redundancy from the perspective 
  of number of bedrooms. Another way to think about it is removing house 
  size will be more detrimental to model performance than removing number 
  of bedrooms, as house size can better compensate for the absence of 
  number of bedrooms. This also implies that house size would be a more 
  important feature than number of bedrooms in the model. *The important 
  point here is that like synergy, redundancy is a naturally asymmetric 
  property of the global information feature pairs have for predicting 
  an outcome.* Redundancy is expressed as a percentage ranging from 0% 
  (full uniqueness) to 100% (full redundancy).


Note that cases can apply at the same time so a feature pair can use some information synergistically and some information redundantly.

The FACET `LearnerInspector` can calculate all of this with a single method call, but also offers methods to access the intermediate results of each step. A lightweight visualization framework is available to render the results in different styles.

SHAP values from the `LearnerInspector` can also be used with the SHAP package plotting functions for sample and observation level SHAP visualizations, such as SHAP distribution plots, dependency plots, force plots and waterfall plots.

In [ ]:
# run inspector
clf_inspector = LearnerInspector(
    n_jobs=-3,
    verbose=False,
).fit(crossfit=clf_ranker.best_model_crossfit_)

## Feature importance

Feature importance has many ways of being measured. Here we utilize the FACET implementation based on the `LearnerInspector`. Each feature is ranked according to the mean SHAP value for that feature. This plot is paired with a standard SHAP distribution plot for features to see if there is any directional tendency for the associations.

In [ ]:
# FACET feature importance
f_importance = clf_inspector.feature_importance()
plt.subplot(1, 2, 1)
f_importance.sort_values().plot.barh()

# get some info for standard SHAP model inspection
shap_data = clf_inspector.shap_plot_data()

# standard SHAP summary plot using the shap package
plt.subplot(1, 2, 2)
shap.summary_plot(shap_values=shap_data.shap_values, features=shap_data.features, show=False, plot_size=(16.0, 8.0))
plt.tight_layout()

Based on the feature importance's we can see the top five features are age, RBC count, waist to height ratio, average systolic blood pressure and waist circumference. Inspection of the SHAP value distributions does not provide any indication of a general direction of association for any features.

## Synergy

In [ ]:
# synergy heatmap
synergy_matrix = clf_inspector.feature_synergy_matrix()
MatrixDrawer(style="matplot%").draw(synergy_matrix, title="Feature synergies")

## Redundancy

In [ ]:
# redundancy heatmap
redundancy_matrix = clf_inspector.feature_redundancy_matrix()
MatrixDrawer(style="matplot%").draw(redundancy_matrix, title="Feature redundancies")

## Feature clustering

As detailed above redundancy and synergy for a feature pair is from the "perspective" of one of the features in the pair, and so yields two distinct values. However, a symmetric version can also be computed that provides not only a simplified perspective but allows the use of (1 - metric) as a feature distance. With this distance hierarchical, single linkage clustering is applied to create a dendrogram visualization. This helps to identify groups of low distance, features which activate "in tandem" to predict the outcome. Such information can then be used to either reduce clusters of highly redundant features to a subset or highlight clusters of highly synergistic features that should always be considered together.

For this example, let's apply clustering to redundancy to see how the apparent grouping observed in the heatmap appears in the dendrogram. Ideally, we want to see features only start to cluster as close to the right-hand side of the dendrogram as possible. This implies all features in the model are contributing uniquely to our predictions.

In [ ]:
# redundancy dendrogram
dd_redundancy = clf_inspector.feature_redundancy_linkage()
DendrogramDrawer().draw(title="Redundancy linkage", data=dd_redundancy)

For convenience when working in a non-notebook environment, all of the `Drawer`'s provided by the [pytools](https://github.com/BCG-Gamma/pytools) package also support a `style='text'` flag.

In [ ]:
DendrogramDrawer(style="text").draw(title="Redundancy linkage", data=dd_redundancy)

## Removing redundant features

Recall the redundancy dendrogram above where we saw a clear cluster of features with redundancy; 

- assess if the features of the model are unique, i.e. not redundant with other features
- decide which features to discard, combine, or modify to increase the uniqueness of important features in the model



In [ ]:
# drop redundant features from our FACET sample object
shop_no_redundant_feat = shop_initial_features.drop(
    feature_names=["ExitRates", "Month"]
)

In [ ]:
# re-run ranker without redundant features
clf_ranker = LearnerRanker(
    grids=classifier_grid,
    cv=RepeatedKFold(n_splits=5, n_repeats=10, random_state=42),
    n_jobs=-3,
    scoring="roc_auc",
).fit(shop_no_redundant_feat)

clf_ranker.summary_report()

In [ ]:
# run inspector
inspector_no_redun = LearnerInspector(
    n_jobs=-3,
    verbose=False,
).fit(crossfit=clf_ranker.best_model_crossfit_)

In [ ]:
# redundancy dendrogram
dd_redundancy = inspector_no_redun.feature_redundancy_linkage()
DendrogramDrawer().draw(title="HD set features", data=dd_redundancy)

Now with the removal of `BMI` and `Waist Circumference` we can see the feature clustering starts much further to the right.

We can also check the best ranked model after removing redundant features.

In [ ]:
clf_ranker.best_model_.classifier